In [ ]:
import zipfile
import os

def unzip_file(zip_filepath, extract_to_dir):
    """
    Unzips a zip file to the specified directory.

    :param zip_filepath: Path to the zip file
    :param extract_to_dir: Directory where the files should be extracted
    """
    # Ensure the extraction directory exists
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)

    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
        print(f"Extracted all files to {extract_to_dir}")

# Example usage
zip_filepath = '/content/drive/MyDrive/Input_Segment_Images.zip'
extract_to_dir = '/content/Input_Segment_Images'
unzip_file(zip_filepath, extract_to_dir)



Extracted all files to /content/Input_Segment_Images


In [ ]:
!pip install roboflow ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 890.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [ ]:
!pip install inference-sdk==0.9.9
from inference_sdk import InferenceHTTPClient

# create an inference client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="....."
)

# run inference on a local image
print(CLIENT.infer(
    "/content/Input_Segment_Images/Input_Segment_Images/000001.jpg",
    model_id="smoke-annotation-h9ghb/1"
))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00
{'time': 0.0159493769999699, 'image': {'width': 1920, 'height': 1080}, 'predictions': [{'x': 403.125, 'y': 885.9375, 'width': 191.25, 'height': 283.125, 'confidence': 0.592505156993866, 'class': 'smoke', 'class_id': 0, 'detection_id': '64614685-99aa-410b-b62b-dbba612b419d'}]}


In [ ]:
import os
import cv2
from inference_sdk import InferenceHTTPClient
import numpy as np
import requests

# Create an inference client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="W1Sv7zZBCuEElQdqvcAG"
)

# Define input and output directories
input_dir = "/content/Input_Segment_Images/Input_Segment_Images"
output_dir = "/content/drive/MyDrive/Annotation_Prediction"
os.makedirs(output_dir, exist_ok=True)

# Function to draw bounding boxes on an image
def draw_boxes(img, predictions):
    for prediction in predictions:
        x, y, w, h = prediction['x'], prediction['y'], prediction['width'], prediction['height']
        class_name = prediction['class']
        confidence = prediction['confidence']

        x1 = int(x - w / 2)
        y1 = int(y - h / 2)
        x2 = int(x + w / 2)
        y2 = int(y + h / 2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, f"{class_name}: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return img

# Process each image in the input directory
for img_name in os.listdir(input_dir):
    img_path = os.path.join(input_dir, img_name)

    # Read and resize the image
    img = cv2.imread(img_path)
    resized_img = cv2.resize(img, (256, 256))

    # Save the resized image to a temporary file
    temp_img_path = "temp_img.jpg"
    cv2.imwrite(temp_img_path, resized_img)

    # Run inference on the resized image
    response = CLIENT.infer(temp_img_path, model_id="smoke-annotation-h9ghb/1")
    predictions = response['predictions']

    # Draw bounding boxes on the resized image
    annotated_img = draw_boxes(resized_img, predictions)

    # Save the annotated image
    output_path = os.path.join(output_dir, img_name)
    cv2.imwrite(output_path, annotated_img)

# Clean up the temporary file
os.remove(temp_img_path)


In [ ]:
import os
from roboflow import Roboflow
from ultralytics import YOLO
import cv2

# Initialize Roboflow API and download the YOLOv8 model
rf = Roboflow(api_key="W1Sv7zZBCuEElQdqvcAG")
project = rf.workspace("smoke-annotation").project("smoke-annotation-h9ghb")
version = project.version(1)
dataset = version.download("yolov8")

# Load the YOLOv8 model
#model = project.version(VERSION).model
model_path = os.path.join(dataset.location, "weights", "best.pt")
model = YOLO(model_path)

# Define input and output directories
input_dir = "/content/Input_Segment_Images/Input_Segment_Images"
output_dir = "/content/Annotation_Prediction"
os.makedirs(output_dir, exist_ok=True)

# Function to annotate and save images
def annotate_images(input_folder, output_folder, model):
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)

        # Read the image
        img = cv2.imread(img_path)

        # Perform prediction
        results = model.predict(source=img_path)

        # Draw bounding boxes on the image
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy)
                label = box.cls
                confidence = box.conf
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, f"{label}: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Save the annotated image
        output_path = os.path.join(output_folder, img_name)
        cv2.imwrite(output_path, img)

# Annotate images in the input directory and save them to the output directory
annotate_images(input_dir, output_dir, model)
